In [6]:
from custom_libraries.miscellaneous import *
from custom_libraries.image_dataset import *
from custom_libraries.aktree import *
import numpy as np
import gc
from os.path import exists

In [7]:
#############
### SETUP ###
#############

project_folder = ''
bs = 256
trials = 10
epochs = 2000
trees_set = [1]
asymmetry_index = .2
filename = project_folder + 'results/aktree_history_' + str(int(100 * asymmetry_index)) + '_testLearningRate.npy'
classes = [[3, 5, 'mnist'],
           [0, 6, 'fmnist'],
           [14, 17, 'emnist'],
           [2, 6, 'kmnist'],
           [3, 5, 'cifar10'],
           [5, 6, 'svhn'],
           [3, 5, 'usps']]
verbose = 1
learning_rates = [.1, 1]

#################
### END SETUP ###
#################

if exists(filename) and input("Resume computation? (y/n)") == 'y':
    print("Recovering data...")
    history = np.load(filename, allow_pickle=True)
else:
    print("Results will be saved to:", filename)
    history = np.zeros((len(classes), trials, len(trees_set), 3))

for j, (t1, t2, ds) in enumerate(classes):

    # escludo i dataset a colori per il momento
    if ds in ['cifar10', 'svhn']:
        continue

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test', data_dir=None)
    train_ds = ImageDataset(ds, 'train', data_dir=None)

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-tree")

        test_set = tf.data.Dataset.from_tensor_slices((test_ds.images, test_ds.labels)).map(
            lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        for i in range(trials):

            if history[j, i, k, 0] != 0:
                continue

            print(f"Trial {i + 1}")

            with tf.device('/device:GPU:0'):

                X_train, y_train, X_valid, y_valid = train_ds.bootstrap(.85, True)

                train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)).map(
                    lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
                valid_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).map(
                    lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

                tree_structure = create_asymmetric_tree_structure(len(train_ds.images[0]), .2)
                tree_structure.reverse()
                data1 = []

                for learning_rate in learning_rates:

                    for _ in range(2):

                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=2000),
                                     tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints/ktree_orig_checkpoint",
                                                                        monitor='val_binary_crossentropy',
                                                                        verbose=0,
                                                                        save_best_only=True,
                                                                        save_weights_only=True,
                                                                        )]


                        model, history[j, i, k, 2] = create_asymmetric_model(unique_pixels=len(train_ds.images[0]),
                                                                             use_bias=False, non_neg=False,
                                                                             asymmetry_index=asymmetry_index, learning_rate=learning_rate,
                                                                             layer_structure=tree_structure)

                        fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                                validation_data=valid_set, validation_batch_size=bs,
                                                callbacks=callbacks, verbose=verbose)
                        print_fit_history(fit_history, epochs)
                        model.load_weights('checkpoints/ktree_orig_checkpoint')

                        evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
                        print_evaluate_history(evaluate_history)

                        history[j, i, k, 0:2] = evaluate_history[1:]
                        data1.append(evaluate_history[2])

                        #np.save(filename, history, allow_pickle=True)

                        del model
                        gc.collect()

                print(data)

Results will be saved to: results/aktree_history_20_testLearningRate.npy
Dataset: mnist / Pair: 3-5
1-tree
Trial 1
Depth of the tree:  13
Epoch 1/2000
39/39 [==============================] - 24s 53ms/step - loss: 426.9753 - binary_crossentropy: 426.9753 - acc: 0.4813 - val_loss: 0.9724 - val_binary_crossentropy: 0.9724 - val_acc: 0.3756
Epoch 2/2000
39/39 [==============================] - 1s 25ms/step - loss: 0.9410 - binary_crossentropy: 0.9410 - acc: 0.4008 - val_loss: 0.8570 - val_binary_crossentropy: 0.8570 - val_acc: 0.4143
Epoch 3/2000
39/39 [==============================] - 1s 26ms/step - loss: 0.8031 - binary_crossentropy: 0.8031 - acc: 0.4991 - val_loss: 0.7818 - val_binary_crossentropy: 0.7818 - val_acc: 0.4991
Epoch 4/2000
39/39 [==============================] - 1s 26ms/step - loss: 0.7490 - binary_crossentropy: 0.7490 - acc: 0.5141 - val_loss: 0.7209 - val_binary_crossentropy: 0.7209 - val_acc: 0.5586
Epoch 5/2000
39/39 [==============================] - 1s 26ms/step - 

KeyboardInterrupt: 

In [ ]:
print(data)